In [ ]:
import MDAnalysis as mda
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pp
import analyse as ana
from scipy.stats import gaussian_kde as kde
from IPython.display import display, Markdown, Latex
from matplotlib.ticker import FormatStrFormatter
%matplotlib widget

In [ ]:
from matplotlib.font_manager import FontProperties

textwidth = 6.50128 # latex textwidth

font0 = FontProperties()

font = font0.copy()
font.set_family('sans-serif')
font.set_style('normal')
font.set_weight('bold')
font.set_size(16)

Title = font.copy()
Title.set_size(18)

font2 = Title.copy()
font2.set_size(12)

font3 = font.copy()
font3.set_size(12)
font3.set_weight('normal')

mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['font.size'] = 12
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['axes.labelweight'] = 'bold'
mpl.rcParams['font.family'] = 'sans-serif'

mpl.rcParams['figure.figsize'] = (textwidth, 4)

In [ ]:
### Plot

def plot(unis, inputs):
    fig, axes = pp.subplots(len(inputs) // 2, 2, sharex=True, sharey=False)
    #fig.suptitle("Probability distribution of distances (in Angstroms)")   


    fig.set_size_inches(textwidth, 4 * len(inputs) // 4)
    fig.subplots_adjust(
        wspace=0.05, 
        hspace=0.25,
        right=0.95,
        left=0.05,
        top=0.95,
        bottom=0.05
    )
    for n, inp in enumerate(inputs):
        ax = axes.flatten()[n]
        ax.set_title(inp['dist_label'])
        for par, uni in unis.items():
            #display(Markdown(f'## {par}'))
            sels = []
            for s in inp['selections']:
                sel = s.copy()
                if 'resid' in sel:
                    sel['resid'] -= (par == 'NMR')*resid_offset
                seltxt = ' and '.join(
                    [f'{k} {v}' for k, v in sel.items()]
                )
                sels.append(uni.select_atoms(seltxt))


            dists = uni.get_distances(*sels)
            dists = dists.flatten()

            _, bins = np.histogram(dists)

            distrange = np.linspace(bins[0] - xpad, bins[-1] + xpad, 200)

            kwargs = dict(
                label=par,
                linewidth=2,
            )
            if par == 'NMR': 
                kwargs['color'] = 'k'
                kwargs['linestyle'] = '--'

            lines = ax.plot(distrange, kde(dists)(distrange), **kwargs)

            mean = np.mean(dists)
            kwargs['color'] = lines[-1].get_color()
            kwargs['label'] = None
            ax.axvline(mean, **kwargs)
        ax.set_ylim(0, None)
        if n%2:
            ax.yaxis.tick_right()
            
    return fig, axes

In [ ]:
resid_offset = 1 # output (Simulation) resid minus input (NMR) resid

### Parameter sets

parsets = [
   'CTPOL', 'opt-CTPOL'
]

ref = 'trajectories/input.pdb'

### Inputs for angle calculating function

inputs = [
    dict(
        dist_label = "S4-Zn",
        selections = [
            dict(name='SG', resid=4),
            dict(name='ZN')
        ]
    ),
    dict(
        dist_label = "S7-Zn",
        selections = [
            dict(name='SG', resid=7),
            dict(name='ZN')
        ]
    ),
    dict(
        dist_label = "N20-Zn",
        selections = [
            dict(name='NE2', resid=20),
            dict(name='ZN')
        ]
    ),
    dict(
        dist_label = "N24-Zn",
        selections = [
            dict(name='NE2', resid=24),
            dict(name='ZN')
        ]
    ),
    dict(
        dist_label = "S4-S7",
        selections = [
            dict(name='SG', resid=4),
            dict(name='SG', resid=7),
        ]
    ),
    dict(
        dist_label = "N20-N24",
        selections = [
            dict(name='NE2', resid=20),
            dict(name='NE2', resid=24),
        ]
    ),
    dict(
        dist_label = "N20-S7",
        selections = [
            dict(name='NE2', resid=20),
            dict(name='SG', resid=7)
        ]
    ),
    dict(
        dist_label = "N24-S7",
        selections = [
            dict(name='NE2', resid=24),
            dict(name='SG', resid=7)
        ]
    )
]

xpad = 0.1

### Get Universes


unis = dict(NMR=ana.Universe(ref, ref))
for par in parsets:
    dcd = f"trajectories/MD-1ZNF/{par}/centered_output.dcd"
    unis[par] = ana.Universe('output.pdb', dcd)

In [ ]:
fig, axes = plot(unis, inputs)


In [ ]:
pp.gca().set_xlim(1.5,3.95)
axes[-1][-1].legend()
fig

In [ ]:
fig.savefig("Images/Distance_distributions.pdf", dpi=300)

# Cross distances (S-S, N-N, etc)

In [ ]:
inputs = [
    dict(
        dist_label = "S4-S7",
        selections = [
            dict(name='SG', resid=4),
            dict(name='SG', resid=7),
        ]
    ),
    dict(
        dist_label = "N20-N24",
        selections = [
            dict(name='NE2', resid=20),
            dict(name='NE2', resid=24),
        ]
    ),
    dict(
        dist_label = "N20-S7",
        selections = [
            dict(name='NE2', resid=20),
            dict(name='SG', resid=7)
        ]
    ),
    dict(
        dist_label = "N24-S7",
        selections = [
            dict(name='NE2', resid=24),
            dict(name='SG', resid=7)
        ]
    )
]

In [ ]:
fig, axes = plot(unis, inputs)

In [ ]:
pp.gca().set_xlim(2.55,4)
axes[0][0].xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

In [ ]:
axes[0][1].legend()


In [ ]:
fig.savefig("Distance_distributions_cross.png")